### **ENVIRONMENT SETUP**

In [2]:
%cd /kaggle/working/
! rm -rf emotion-classification
! git clone https://github.com/srivarshan-s/emotion-classification.git
%cd emotion-classification

/kaggle/working
Cloning into 'emotion-classification'...
remote: Enumerating objects: 120, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 120 (delta 0), reused 5 (delta 0), pack-reused 113
Receiving objects: 100% (120/120), 101.64 MiB | 33.10 MiB/s, done.
Resolving deltas: 100% (7/7), done.
/kaggle/working/emotion-classification


### **IMPORT LIBRARIES**

In [3]:
import os
import re
import string
import json
import emoji
import warnings
warnings.filterwarnings('ignore')

import pickle
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option("display.max_columns", None)

from sklearn import metrics
from bs4 import BeautifulSoup
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, AutoTokenizer, BertModel, BertConfig, AutoModel, AdamW
import tensorflow as tf
import tensorflow.keras as keras
import pickle

### **LOAD DATASET**

In [4]:
# Load the testing and validations datasets

df_train = pd.read_csv("data/train.tsv", sep='\t', header=None, names=['Text', 'Class', 'ID'])
df_dev = pd.read_csv("data/dev.tsv", sep='\t', header=None, names=['Text', 'Class', 'ID'])

In [5]:
df_train.head()

,Text,Class,ID
0,My favourite food is anything I didn't have to...,27,eebbqej
1,"Now if he does off himself, everyone will thin...",27,ed00q6i
2,WHY THE FUCK IS BAYLESS ISOING,2,eezlygj
3,To make her feel threatened,14,ed7ypvh
4,Dirty Southern Wankers,3,ed0bdzj


In [6]:
# Obtain the Class List and Number of Classes

df_train['List of classes'] = df_train['Class'].apply(lambda x: x.split(','))
df_train['Len of classes'] = df_train['List of classes'].apply(lambda x: len(x))

df_dev['List of classes'] = df_dev['Class'].apply(lambda x: x.split(','))
df_dev['Len of classes'] = df_dev['List of classes'].apply(lambda x: len(x))

In [7]:
df_train.head()

,Text,Class,ID,List of classes,Len of classes
0,My favourite food is anything I didn't have to...,27,eebbqej,[27],1
1,"Now if he does off himself, everyone will thin...",27,ed00q6i,[27],1
2,WHY THE FUCK IS BAYLESS ISOING,2,eezlygj,[2],1
3,To make her feel threatened,14,ed7ypvh,[14],1
4,Dirty Southern Wankers,3,ed0bdzj,[3],1


In [8]:
# Load the ekman mapping

with open('data/ekman_mapping.json') as file:
    ekman_mapping = json.load(file)
    
ekman_mapping

{'anger': ['anger', 'annoyance', 'disapproval'],
 'disgust': ['disgust'],
 'fear': ['fear', 'nervousness'],
 'joy': ['joy',
  'amusement',
  'approval',
  'excitement',
  'gratitude',
  'love',
  'optimism',
  'relief',
  'pride',
  'admiration',
  'desire',
  'caring'],
 'sadness': ['sadness', 'disappointment', 'embarrassment', 'grief', 'remorse'],
 'surprise': ['surprise', 'realization', 'confusion', 'curiosity']}

In [9]:
# Load the list of original emotions

emotion_file = open("data/emotions.txt", "r")
emotion_list = emotion_file.read()
emotion_list = emotion_list.split("\n")
print(emotion_list)
print(f"Number of Original Emotions = {len(emotion_list)}")

['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral']
Number of Original Emotions = 28


In [10]:
# Define function to convert class number to emotion

def idx2class(idx_list):
    arr = []
    for i in idx_list:
        arr.append(emotion_list[int(i)])
    return arr

In [11]:
df_train['Emotions'] = df_train['List of classes'].apply(idx2class)
df_dev['Emotions'] = df_dev['List of classes'].apply(idx2class)

In [12]:
df_train.head()

,Text,Class,ID,List of classes,Len of classes,Emotions
0,My favourite food is anything I didn't have to...,27,eebbqej,[27],1,[neutral]
1,"Now if he does off himself, everyone will thin...",27,ed00q6i,[27],1,[neutral]
2,WHY THE FUCK IS BAYLESS ISOING,2,eezlygj,[2],1,[anger]
3,To make her feel threatened,14,ed7ypvh,[14],1,[fear]
4,Dirty Southern Wankers,3,ed0bdzj,[3],1,[annoyance]


In [13]:
# Define function to map the emotions according to the ekman mapping

def EmotionMapping(emotion_list):
    map_list = []
    for i in emotion_list:
        if i in ekman_mapping['anger']:
            map_list.append('anger')
        if i in ekman_mapping['disgust']:
            map_list.append('disgust')
        if i in ekman_mapping['fear']:
            map_list.append('fear')
        if i in ekman_mapping['joy']:
            map_list.append('joy')
        if i in ekman_mapping['sadness']:
            map_list.append('sadness')
        if i in ekman_mapping['surprise']:
            map_list.append('surprise')
        if i == 'neutral':
            map_list.append('neutral')
    return map_list

In [14]:
df_train['Mapped Emotions'] = df_train['Emotions'].apply(EmotionMapping)
df_dev['Mapped Emotions'] = df_dev['Emotions'].apply(EmotionMapping)

In [15]:
df_train.head()

,Text,Class,ID,List of classes,Len of classes,Emotions,Mapped Emotions
0,My favourite food is anything I didn't have to...,27,eebbqej,[27],1,[neutral],[neutral]
1,"Now if he does off himself, everyone will thin...",27,ed00q6i,[27],1,[neutral],[neutral]
2,WHY THE FUCK IS BAYLESS ISOING,2,eezlygj,[2],1,[anger],[anger]
3,To make her feel threatened,14,ed7ypvh,[14],1,[fear],[fear]
4,Dirty Southern Wankers,3,ed0bdzj,[3],1,[annoyance],[anger]


In [16]:
# Create columns for the new mapped emotions

df_train['anger'] = np.zeros((len(df_train),1))
df_train['disgust'] = np.zeros((len(df_train),1))
df_train['fear'] = np.zeros((len(df_train),1))
df_train['joy'] = np.zeros((len(df_train),1))
df_train['sadness'] = np.zeros((len(df_train),1))
df_train['surprise'] = np.zeros((len(df_train),1))
df_train['neutral'] = np.zeros((len(df_train),1))

df_dev['anger'] = np.zeros((len(df_dev),1))
df_dev['disgust'] = np.zeros((len(df_dev),1))
df_dev['fear'] = np.zeros((len(df_dev),1))
df_dev['joy'] = np.zeros((len(df_dev),1))
df_dev['sadness'] = np.zeros((len(df_dev),1))
df_dev['surprise'] = np.zeros((len(df_dev),1))
df_dev['neutral'] = np.zeros((len(df_dev),1))

# Fill the column with the corresponding emotions
for i in ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise','neutral']:
    df_train[i] = df_train['Mapped Emotions'].apply(lambda x: 1 if i in x else 0)
    df_dev[i] = df_dev['Mapped Emotions'].apply(lambda x: 1 if i in x else 0)

In [17]:
df_train.head()

,Text,Class,ID,List of classes,Len of classes,Emotions,Mapped Emotions,anger,disgust,fear,joy,sadness,surprise,neutral
0,My favourite food is anything I didn't have to...,27,eebbqej,[27],1,[neutral],[neutral],0,0,0,0,0,0,1
1,"Now if he does off himself, everyone will thin...",27,ed00q6i,[27],1,[neutral],[neutral],0,0,0,0,0,0,1
2,WHY THE FUCK IS BAYLESS ISOING,2,eezlygj,[2],1,[anger],[anger],1,0,0,0,0,0,0
3,To make her feel threatened,14,ed7ypvh,[14],1,[fear],[fear],0,0,1,0,0,0,0
4,Dirty Southern Wankers,3,ed0bdzj,[3],1,[annoyance],[anger],1,0,0,0,0,0,0


In [18]:
df_dev.head()

,Text,Class,ID,List of classes,Len of classes,Emotions,Mapped Emotions,anger,disgust,fear,joy,sadness,surprise,neutral
0,Is this in New Orleans?? I really feel like th...,27,edgurhb,[27],1,[neutral],[neutral],0,0,0,0,0,0,1
1,"You know the answer man, you are programmed to...","4,27",ee84bjg,"[4, 27]",2,"[approval, neutral]","[joy, neutral]",0,0,0,1,0,0,1
2,I've never been this sad in my life!,25,edcu99z,[25],1,[sadness],[sadness],0,0,0,0,1,0,0
3,The economy is heavily controlled and subsidiz...,"4,27",edc32e2,"[4, 27]",2,"[approval, neutral]","[joy, neutral]",0,0,0,1,0,0,1
4,He could have easily taken a real camera from ...,20,eepig6r,[20],1,[optimism],[joy],0,0,0,1,0,0,0


In [19]:
# Drop rows with more than 1 emotion

df_train.drop(df_train[df_train["Len of classes"] > 1].index, axis=0, inplace=True)
df_dev.drop(df_dev[df_dev["Len of classes"] > 1].index, axis=0, inplace=True)

In [20]:
# Drop rows with neutral and disgust emotions

# df_train.drop(df_train[df_train['neutral'] == 1].index, inplace=True)
# df_dev.drop(df_dev[df_dev['neutral'] == 1].index, inplace=True)
# df_train.drop(df_train[df_train['disgust'] == 1].index, inplace=True)
# df_dev.drop(df_dev[df_dev['disgust'] == 1].index, inplace=True)

In [21]:
# Drop extra columns

# df_train.drop(['Class', 'List of classes', 'Len of classes', 'Emotions', 'Mapped Emotions', 'neutral', 'disgust'], axis=1, inplace=True)
# df_dev.drop(['Class', 'List of classes', 'Len of classes', 'Emotions', 'Mapped Emotions', 'neutral', 'disgust'], axis=1, inplace=True)

df_train.drop(['Class', 'List of classes', 'Len of classes', 'Emotions', 'Mapped Emotions'], axis=1, inplace=True)
df_dev.drop(['Class', 'List of classes', 'Len of classes', 'Emotions', 'Mapped Emotions'], axis=1, inplace=True)

In [22]:
df_train.head()

,Text,ID,anger,disgust,fear,joy,sadness,surprise,neutral
0,My favourite food is anything I didn't have to...,eebbqej,0,0,0,0,0,0,1
1,"Now if he does off himself, everyone will thin...",ed00q6i,0,0,0,0,0,0,1
2,WHY THE FUCK IS BAYLESS ISOING,eezlygj,1,0,0,0,0,0,0
3,To make her feel threatened,ed7ypvh,0,0,1,0,0,0,0
4,Dirty Southern Wankers,ed0bdzj,1,0,0,0,0,0,0


### **DATA PREPROCESSING**

In [23]:
# Mapping for expansion of contracted words

contraction_mapping = {
    "ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", 
    "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", 
    "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", 
    "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", 
    "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", 
    "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", 
    "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", 
    "it'll've": "it will have", "it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", 
    "might've": "might have","mightn't": "might not", "mightn't've": "might not have", "must've": "must have", 
    "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have",
    "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
    "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", 
    "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", 
    "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", 
    "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", 
    "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is",
    "they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", 
    "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", 
    "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", 
    "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
    "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did",
    "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", 
    "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", 
    "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", 
    "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would", "y'all'd've": "you all would have",
    "y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have",
    "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", 'u.s':'america', 
    'e.g':'for example'
}

# Function to expand the contractions

def clean_contractions(text, mapping):
    '''Clean contraction using contraction mapping'''    
    # Uniform apostrophe
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
        
    # Expand the words using mapping
    for word in mapping.keys():
        if ""+word+"" in text:
            text = text.replace(""+word+"", ""+mapping[word]+"")
            
    # Remove Punctuations
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text) # Create space b/w word and punctuation following it
    text = re.sub(r"([?.!,¿])", r" \1 ", text)
    text = re.sub(r'[" "]+', " ", text)
    return text

In [24]:
# List of punctuations

punct = [
    ',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',
    '~', '@', '£', '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', 
    '½', 'à', '…', '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', 
    '—', '‹', '─', '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', 
    '⋅', '‘', '∞', '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', 
    '‡', '√', 
]

# Mapping for punctuations

punct_mapping = {
    "‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-",
    "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 
    'β': 'beta', '∅': '', '³': '3', 'π': 'pi', '!':' '
}

In [25]:
# Dictionary to correct the misspellings

mispell_dict = {
    'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 
    'theatre': 'theater', 'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 
    'citicise': 'criticize', 'youtu ': 'youtube ', 'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 
    'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can', 'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 
    'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 'mastrubation': 'masturbation', 'mastrubate': 'masturbate', 
    "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'Ethereum', 'narcissit': 'narcissist', 'bigdata': 'big data', 
    '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 'airhostess': 'air hostess', "whst": 'what', 
    'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization', 'demonetisation': 'demonetization'
}

# Function to correct the misspellings

def correct_spelling(x, dic):
    for word in dic.keys():
        x = x.replace(word, dic[word])
    return x

In [26]:
# Function to clean the text

def clean_text(text):
    text = emoji.demojize(text) # Remove emojis
    text = re.sub(r'\:(.*?)\:','',text) 
    text = str(text).lower() # Make text lowercase
    text = re.sub('\[.*?\]', '', text) # Remove text inside square brackets
    text = BeautifulSoup(text, 'lxml').get_text() # Remove html
    text = re.sub('https?://\S+|www\.\S+', '', text) # Remove links
    text = re.sub('<.*?>+', '', text) # Remove text inside angular brackets
    text = re.sub('\n', '', text) # Remove newlines
    text = re.sub('\w*\d\w*', '', text) # Remove words that contain numbers
    # Replace everything with space except (a-z, A-Z, ".", "?", "!", ",", "'")
    text = re.sub(r"[^a-zA-Z?.!,¿']+", " ", text)
    return text

In [27]:
# Function to remove any special characters

def clean_special_chars(text, punct, mapping):
    for p in mapping:
        text = text.replace(p, mapping[p])
    for p in punct:
        text = text.replace(p, f' {p} ')
    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''} # List of special characters
    for s in specials:
        text = text.replace(s, specials[s])
    return text

# Function to remove any extra spaces

def remove_space(text):
    text = text.strip()
    text = text.split()
    return " ".join(text)

In [28]:
# Function to perform the entire preprocessing

def text_preprocessing_pipeline(text):
    text = clean_text(text)
    text = clean_contractions(text, contraction_mapping)
    text = clean_special_chars(text, punct, punct_mapping)
    text = correct_spelling(text, mispell_dict)
    text = remove_space(text)
    return text

In [29]:
# Apply preprocessing

df_train['Text'] = df_train['Text'].apply(text_preprocessing_pipeline)
df_dev['Text'] = df_dev['Text'].apply(text_preprocessing_pipeline)

In [30]:
# Reset the Indices

df_train = df_train.reset_index(drop=True)
df_dev = df_dev.reset_index(drop=True)

In [31]:
df_train.head()

,Text,ID,anger,disgust,fear,joy,sadness,surprise,neutral
0,my favorite food is anything i did not have to...,eebbqej,0,0,0,0,0,0,1
1,now if he does off himself everyone will think...,ed00q6i,0,0,0,0,0,0,1
2,why the fuck is bayless isoing,eezlygj,1,0,0,0,0,0,0
3,to make her feel threatened,ed7ypvh,0,0,1,0,0,0,0
4,dirty southern wankers,ed0bdzj,1,0,0,0,0,0,0


In [32]:
df_dev.head()

,Text,ID,anger,disgust,fear,joy,sadness,surprise,neutral
0,is this in new orleans i really feel like this...,edgurhb,0,0,0,0,0,0,1
1,i have never been this sad in my life,edcu99z,0,0,0,0,1,0,0
2,he could have easily taken a real camera from ...,eepig6r,0,0,0,1,0,0,0
3,thank you for your vote of confidence but we s...,eczm50f,0,0,0,1,0,0,0
4,wah mum other people call me on my bullshit an...,ed4yr9r,1,0,0,0,0,0,0


In [33]:
print(df_train.shape)
print(df_dev.shape)

(36308, 9)
(4548, 9)


### **PRE-CONFIG FOR BERT**

In [34]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [35]:
MAX_LEN = 200
TRAIN_BATCH_SIZE = 64
VALID_BATCH_SIZE = 64
EPOCHS = 10
LEARNING_RATE = 2e-5

### **BUILD DATASET FOR BERT**

In [36]:
# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained('roberta-base')

# Extract Target Labels
target_labels = [col for col in df_train.columns if col not in ['Text', 'ID']]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [37]:
print(f"The target labels are : {target_labels}")

The target labels are : ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'neutral']


In [38]:
# Define class for dataset

class BertDataset(Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.df = df
        self.max_len = max_len
        self.text = df.Text
        self.tokenizer = tokenizer
        self.targets = df[target_labels].values
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        text = self.text[index]
        inputs = self.tokenizer.encode_plus(
            text,
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]
        
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [39]:
# Create Dataset for both training and validation

train_dataset = BertDataset(df_train, tokenizer, MAX_LEN)
valid_dataset = BertDataset(df_dev, tokenizer, MAX_LEN)

In [40]:
# Create DataLoader for training and validation

train_loader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, 
                          num_workers=4, shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=VALID_BATCH_SIZE, 
                          num_workers=4, shuffle=False, pin_memory=True)

### **BUILD CUSTOM MODEL**

In [41]:
# Create class for custom model

class CustomModel(torch.nn.Module):
    
    def __init__(self):
        super(CustomModel, self).__init__()
        self.roberta = AutoModel.from_pretrained('roberta-base') # Load RoBerta model
#         self.fc = torch.nn.Linear(768,5) # Connect to fully-connected layer for output
        self.fc = torch.nn.Linear(768,7) # Connect to fully-connected layer for output
    
    def forward(self, ids, mask, token_type_ids):
        _, features = self.roberta(ids, attention_mask = mask, 
                                   token_type_ids = token_type_ids, return_dict=False)
        output = self.fc(features)
        return output

In [42]:
# Load custom model

model = CustomModel().to(device)

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### **TRAIN CUSTOM BERT MODEL**

In [43]:
loss_fn = torch.nn.BCEWithLogitsLoss() # Loss function
optimizer = AdamW(params =  model.parameters(), lr=LEARNING_RATE, weight_decay=1e-6) # Optimizer

In [44]:
# Define function to train the model

def train(epoch):
    
    model.train()
    
    for _, data in enumerate(train_loader, 0):
        
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        loss = loss_fn(outputs, targets)
        
#         if _ % 500 == 0:
#             print(f'Epoch: {epoch + 1}, Loss:  {loss.item()}')
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
    print(f'Epoch: {epoch + 1}, Loss:  {loss.item()}')

In [45]:
# Perform model training

for epoch in range(EPOCHS):
# for epoch in range(1):
    train(epoch)

Epoch: 1, Loss:  0.20336589217185974
Epoch: 2, Loss:  0.24591010808944702
Epoch: 3, Loss:  0.23236705362796783
Epoch: 4, Loss:  0.11677656322717667
Epoch: 5, Loss:  0.14026693999767303
Epoch: 6, Loss:  0.10517556220293045
Epoch: 7, Loss:  0.11188917607069016
Epoch: 8, Loss:  0.16031482815742493
Epoch: 9, Loss:  0.013923891820013523
Epoch: 10, Loss:  0.08216524124145508


### **EVALUATE CUSTOM BERT MODEL**

In [46]:
# Define function for model evaluation

def validation(data_loader):
    
    model.eval()
    fin_targets = []
    fin_outputs = []
    
    with torch.no_grad():
        
        for _, data in enumerate(data_loader, 0):
            
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            
            outputs = model(ids, mask, token_type_ids)
            
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
            
    return fin_outputs, fin_targets

In [47]:
# Perform model evaluation

outputs, targets = validation(valid_loader)

outputs = np.array(outputs) >= 0.5

accuracy = metrics.accuracy_score(targets, outputs)
f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
f1_score_macro = metrics.f1_score(targets, outputs, average='macro')

print(f"Accuracy Score = {accuracy}")
print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

print(metrics.classification_report(targets, outputs))

Accuracy Score = 0.6539138082673702
F1 Score (Micro) = 0.6688275399575276
F1 Score (Macro) = 0.5935260378242055
              precision    recall  f1-score   support

           0       0.51      0.53      0.52       485
           1       0.42      0.44      0.43        61
           2       0.70      0.64      0.67        66
           3       0.79      0.84      0.81      1668
           4       0.53      0.63      0.58       241
           5       0.54      0.52      0.53       435
           6       0.69      0.56      0.62      1592

   micro avg       0.68      0.66      0.67      4548
   macro avg       0.60      0.59      0.59      4548
weighted avg       0.68      0.66      0.67      4548
 samples avg       0.66      0.66      0.66      4548



In [48]:
# Perform model evaluation on training data

outputs, targets = validation(train_loader)

outputs = np.array(outputs) >= 0.5

accuracy = metrics.accuracy_score(targets, outputs)
f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
f1_score_macro = metrics.f1_score(targets, outputs, average='macro')

print(f"Accuracy Score = {accuracy}")
print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

print(metrics.classification_report(targets, outputs))

Accuracy Score = 0.9717693070397708
F1 Score (Micro) = 0.9758064516129032
F1 Score (Macro) = 0.9637510580383475
              precision    recall  f1-score   support

           0       0.96      0.97      0.97      3878
           1       0.91      0.92      0.91       498
           2       0.98      0.96      0.97       515
           3       0.98      0.99      0.99     12920
           4       0.95      0.98      0.97      2121
           5       0.96      0.96      0.96      3553
           6       0.99      0.96      0.98     12823

   micro avg       0.98      0.97      0.98     36308
   macro avg       0.96      0.96      0.96     36308
weighted avg       0.98      0.97      0.98     36308
 samples avg       0.97      0.97      0.97     36308



### **SAVE MODEL**

In [49]:
pickle.dump(model, open("../cust_bert_model.pkl", "wb"))
pickle.dump(model, open("cust_bert_model.pkl", "wb"))